In [1]:
!git clone https://github.com/AliaksandrSiarohin/first-order-model

fatal: destination path 'first-order-model' already exists and is not an empty directory.


In [2]:
cd first-order-model

/content/first-order-model


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [11]:
!ffmpeg -i /content/gdrive/My\ Drive/Colab\ Notebooks/IMG_2162.mp4 -filter:v "crop=720:720" -async 1 test_correct.mp4

ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --ena

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

source_image = imageio.imread('/content/gdrive/My Drive/Colab Notebooks/obama-website2-600x368.jpg')
#driving_video = imageio.get_reader('/content/gdrive/My Drive/Colab Notebooks/IMG_2164.mp4')
driving_video = imageio.get_reader('test_correct.mp4')


ims = []
try:
    for im in driving_video:
        im = (resize(im, (256,256))[..., :3])
        ims.append(im)
except RuntimeError:
    pass

#Resize image and video to 256x256
print("loaded videos")
source_image = resize(source_image, (256, 256))[..., :3]
#driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani
    

HTML(display(source_image, ims).to_html5_video())

In [0]:
from demo import load_checkpoints
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path='/content/gdrive/My Drive/first-order-motion-model/vox-cpk.pth.tar')


In [17]:
from demo import make_animation
from skimage import img_as_ubyte

source_image = imageio.imread('/content/gdrive/My Drive/Colab Notebooks/264px-Official_portrait_of_Barack_Obama.jpg')
source_image = resize(source_image, (256, 256))[..., :3]
predictions = make_animation(source_image, ims, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, ims, predictions).to_html5_video())

100%|██████████| 278/278 [00:33<00:00,  8.27it/s]
